## 단어 시퀀스

해당 단어가 나타났을때 다음 단어를 예측하면서 흘러가는 것

```
p(사건|조건) = p(조건, 사건) == `<모델>` / p(조건) 
```

으로 출현 확률을 계산하는데 이걸 문장 길이에 따라서 n 번 반복

ex)

```
p(운전|난폭) = p(난폭, 운전) / p(난폭)
```

일반적인 좌 -> 우 방향으로 다음 단어 예측을 순방향, 반대는 역방향 언어 모델

p(w|context `주변 맥락 정보`) 로 사용하기도 함

- 마스크 언어 모델

완전 문장 데이터에서 일부 단어를 마스킹 처리 시키면서 학습

- 스킵-그램 모델

중심 단어 주위의 단어를 이용해 학습

## 트랜스포머

### 인코더 - 디코더

기계 번역에서 주로 사용되어지는 방식

입력 데이터를 단어 시퀀스를 이용해 주요 정보를 압축 한 다음 그 정보를 바탕으로 디코더에 타겟 시퀀스 데이터와 함께 전달하여 변환된 결과를 복원하는 방식

ex)

한글로 된

```
나는 26살 입니다.
```

를 인코더에 넣고 디코더엔 시퀀스 시작 키 `<s>` 를 넣으면 

처음은 결과가 `I'm`

그 다음에도 똑같이 인코더에 

```
나는 26살 입니다.
```

디코더엔 `<s> I'm` 를 넣으면 

`I'm 26` 가 나오는 형식

이런 과정을 반복하여 출현 확률 조정

### 셀프 어텐션

자신에게 중요한 부분을 집중하여 성능 올리는 방식

|어텐션|합성곱 신경망|순환 신경망|셀프 어텐션|
|:------:|:--------:|:------:|:---:|
|RNN 문제점을 해결하기 위해 해당 단어가 어떤 요소에 주목할지를 결정 `카페-cafe` 이런식으로|CNN, 지역 특징 추출, 차례데로 단어들을 넘기며 학습, 대신 커널 크기만큼만 학습하는 단점| RNN, 특징중 하나가 이전에 있었던건 거의 잊어지게 되어지는 특징으로 인해 마지막 단어를 거의 의미 있다고 판단하는 경우 존재| 입력 시퀀스에서 부터 연관관계 연결을 지음 RNN 없이 동작, 디코더 블록 수 만큼 반복, `거기-카페`, `갔었어-카페`|

어떤 요소에 주목해야하는지와 그 요소가 어떤 요소와 연관이 있는지도 알 수 있음

query, key, value 로 이루어짐

문장을 이루는 소스 값을 쿼리,

그 쿼리에서 뻗어나와 같은 문장에 연결관계를 지을 타겟들을 키

키 각각에 연관성 가중치를 값 으로 정의

각각 계산은 행렬 수식

query = (word embedding demension x word num) x W(q)

key = (word embedding demension x word num) x W(k)

value = (word embedding demension x word num) x W(v)

W 값들은 각각 갱신되며 서로 다른 초기값들을 가짐

#### Attention 계산해보기

```
Attension(Q, K, V) = softmax(QK^T / sqrt(d_k)) * V
```

그니까 풀어서 보면

`쿼리` 계산 값과 `전치한 키` 계산을 곱한 걸 차원 수 만큼 제곱근 한 걸로 나누어 주고 softmax 치환한 다음 `가중치` 값과 곱함

QK^T

이때 쿼리와 키간의 문맥적 관계성이 있음

In [8]:
import torch
import numpy as np
from torch.nn.functional import softmax

In [2]:
x = torch.tensor([
    [1.0, 0.0, 1.0, 0.0],
    [0.0, 2.0, 0.0, 2.0],
    [1.0, 1.0, 1.0, 1.0]
])

In [3]:
w_query = torch.tensor([
    [1.0, 0.0, 1.0],
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]
])

In [4]:
w_key = torch.tensor([
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0],
])

In [5]:
w_value = torch.tensor([
    [0.0, 2.0, 0.0],
    [0.0, 3.0, 0.0],
    [1.0, 0.0, 3.0],
    [1.0, 1.0, 0.0]
])

In [6]:
mulQuery = torch.matmul(x, w_query)
mulKey = torch.matmul(x, w_key)
mulValue = torch.matmul(x, w_value)

mulQuery, mulKey, mulValue

(tensor([[1., 0., 2.],
         [2., 2., 2.],
         [2., 1., 3.]]),
 tensor([[0., 1., 1.],
         [4., 4., 0.],
         [2., 3., 1.]]),
 tensor([[1., 2., 3.],
         [2., 8., 0.],
         [2., 6., 3.]]))

In [7]:
attScore = torch.matmul(mulQuery, mulKey.T)
attScore

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

In [10]:
dimSqrt = np.sqrt(mulKey.shape[-1])
dimSqrt, mulKey.shape, mulKey.shape[-1]

(1.7320508075688772, torch.Size([3, 3]), 3)

In [11]:
attenSoftmax = softmax(attScore / dimSqrt, dim=-1)
attenSoftmax

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

In [12]:
weight = torch.matmul(attenSoftmax, mulValue)
weight

tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])

### Multi-Head Attention

위의 self-attention 을 동시에 여러번 하는 것

즉 같은 문서를 읽더라도 여러 다른 사람들이 각자 읽어서 연산을 진행함

행렬의 크기는 

`입력 단어 수` x `value 차원 수` => Z

가 되어지며 이걸 여러 다른 사람들 즉 헤더들의 수 만큼 확장

scaleUp(Z, `헤더 수`)

여기서 임의 가중치 행렬 W 을 곱하는데 크기는 `self-attention` 행렬의 열 크기 x 목표 차원 수

최종적으론 `입력 단어 수` x `목표 차원 수`

### Feedforward neural network

(입력 x 가중치) + ... 해당 레이어에 있는 노드 갯수 만큼 n번 반복 + 바이어스 => 계산 값을 활성함수에 전달 => 결과 값을 또 다른 레이어의 노드에 전달

In [2]:
import torch

In [4]:
x = torch.tensor([2, 1])
w1 = torch.tensor([[3, 2, -4], [2, -3, 1]])
b1 = 1
w2 = torch.tensor([[-1, 1], [1, 2], [3, 1]])
b2 = -1

$\begin{bmatrix}2&1\\ \end{bmatrix}$ x $\begin{bmatrix}3&2&-4\\2&-3&1\\ \end{bmatrix}$ + b1 = $\begin{bmatrix}9&2&-6\\ \end{bmatrix}$

```
=> relu =>
```

$\begin{bmatrix}9&2&0\\ \end{bmatrix}$ x $\begin{bmatrix}-1&1\\1&2\\3&1\\ \end{bmatrix}$ + b2 = $\begin{bmatrix}-8&12\\ \end{bmatrix}$

In [6]:
beforeRelu = torch.matmul(x, w1) + b1
beforeRelu

tensor([ 9,  2, -6])

In [7]:
afterRelu = torch.nn.functional.relu(beforeRelu)
afterRelu

tensor([9, 2, 0])

In [8]:
y = torch.matmul(afterRelu, w2) + b2
y

tensor([-8, 12])

### 잔차 연결 residual connection

위 같은 블록 연산을 하는 걸 외에 건너 뛰는 경로를 추가 하는 것

y = F(x) 에 y = F(x) + x 와 같은 꼴

연속적인 결과 값 베이스 연산이 아닌 원본 값을 활용해 가면서 여러 경우의 수로 연산이 가능해 짐

### 레이어 정규화

값들을 너무 들쭉날쭉 하게 하지 말고 균일하게 맞추어 주는 작업

(x - 평균) / 표준편차

In [11]:
x = torch.tensor([[1.0, 2.0, 3.0], [1.0, 1.0, 1.0]])
x.shape, x.shape[-1]

(torch.Size([2, 3]), 3)

In [12]:
m = torch.nn.LayerNorm(x.shape[-1])
m

LayerNorm((3,), eps=1e-05, elementwise_affine=True)

In [14]:
y = m(x)
y

tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<NativeLayerNormBackward0>)

In [15]:
m.weight, m.bias

(Parameter containing:
 tensor([1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], requires_grad=True))

### Drop out

일부 노드를 일부로 꺼주는 작업

학습이 너무 잘 되어 과적합 되어지는 문제 방지

In [21]:
m = torch.nn.Dropout(p=0.2) # 20%

In [22]:
x = torch.randn(1, 10)
x

tensor([[-0.1387, -1.0628,  2.1952,  0.7804,  0.7318,  0.4783,  1.1320,  0.2525,
          0.0367, -0.5461]])

In [23]:
y = m(x)
y

tensor([[-0.0000, -1.3286,  2.7440,  0.9755,  0.9148,  0.5978,  1.4150,  0.0000,
          0.0459, -0.6827]])

노드를 끄는 것만이 아닌 1 / (1 - p) 값을 곱하는 연산도 진행

일반적으론 p 값을 0.1 로 적용

### Adam optimizer

모델과 정답 사이의 오차를 줄여주기 위한 파라미터 수정 작업 

기본적으로 경사 하강법을 사용

이를 최적화라고 부르며 그 방법중 Adam 이란 방식이 있음

내려가는 방향은 경사가 급한 방향으로가지만 관성이 있는것 처럼 일부 방향을 유지함

보폭은 처음 간 곳은 빠르게, 가본 곳은 천천히 움직임

|GPT | BERT |
|:--:|:----:|
|과거 단어들을 이용해 다음 단어 예측| 문장 중간에 마스킹처리를 한 다음 적절한 단어를 예측|
|단방향|양방향|
|문장 생성|의미 추출|

## 파인 튜닝

다운스트림(미리 업스트림에서 학습된 내용 활용) 데이터 전체 활용하여 전체 모델 업데이트

- 프롬포트 튜닝: 데이터 전체 활용, 일부 모델 업데이트
- 인컨텍스트 튜닝: 데이터 일부 사용, 모델 업데이트 x

In [26]:
from transformers import BertTokenizer, BertConfig, BertModel

In [25]:
#https://github.com/Beomi/KcBERT
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False
)

Downloading:   0%|          | 0.00/244k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [27]:
model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)

In [28]:
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=model_config
)

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [30]:
sentences = ["안녕하세요", "하이"]

In [33]:
features = tokenizer(
    sentences,
    max_length=10,
    padding="max_length",
    truncation=True # 길이 범위 초과시 자름
)
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 3, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]}

input_ids 에서 `2`, `3` 은 각각 CLS, SEP 라는 스페셜 토큰임

In [34]:
features_tensor = {k: torch.tensor(v) for k, v in features.items()}
features_tensor

{'input_ids': tensor([[    2, 19017,  8482,     3,     0,     0,     0,     0,     0,     0],
         [    2, 15830,     3,     0,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [35]:
features.items()

dict_items([('input_ids', [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 3, 0, 0, 0, 0, 0, 0, 0]]), ('token_type_ids', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), ('attention_mask', [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])])

In [38]:
for k, v in features.items():
    print("k:", k, "v:", v)

k: input_ids v: [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 3, 0, 0, 0, 0, 0, 0, 0]]
k: token_type_ids v: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
k: attention_mask v: [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]


In [40]:
outputs = model(**features_tensor)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.7565, -1.7148,  2.1903,  ..., -0.4291,  0.9469,  0.7394],
         [ 1.1048, -1.3408,  1.9444,  ...,  0.3340,  0.2276, -0.5220],
         [-0.1317, -0.6982,  1.3086,  ...,  0.4448,  0.0836, -0.6280],
         ...,
         [ 0.0666, -0.7225,  1.8425,  ...,  1.2435,  1.6184,  0.1339],
         [ 0.1781, -0.7823,  1.6892,  ...,  1.1230,  1.5262,  0.1855],
         [-0.0183, -0.8602,  2.5182,  ...,  0.9201,  0.3697, -0.7420]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [41]:
outputs.last_hidden_state.shape

torch.Size([2, 10, 768])

문장 2개의 시퀀스 길이 10, 768차원 벡터